In [20]:
import requests
import csv
import json
import time
from datetime import datetime, timedelta
from collections import defaultdict

api_key = 'mzrsaV5E9ZwVPQ2biOQGyYansI7Ukh5p'


In [21]:
def print_json(json_data):
    print(json.dumps(json_data, indent=2))

def write_jsonl_file(data, filename):
    # Write data to a JSONL file
    with open(filename, 'a', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')


In [22]:
def return_response(url) :
    response = requests.get(url)
    return response.json()

def generate_date_range(start_date, end_date):
    # Generate a date range between start_date and end_date
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(weeks=1)


# Getting All Lists

In [7]:
output_filename = 'all_lists.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    url = f'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={api_key}' # all lists
    response = return_response(url)

    json.dump(response, file, indent=4)
    print(f'All lists saved to {output_filename}.')


All lists saved to all_lists.json.


# Getting All Weeks

In [18]:
start_date_str = '2011-02-13'
end_date_str = '2023-12-10'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

all_responses = {}

output_filename = 'all_weeks.jsonl'
with open(output_filename, 'a', encoding='utf-8') as file:
    for current_date in generate_date_range(start_date, end_date):
        time.sleep(12) #
        week = current_date.strftime('%Y-%m-%d')
        url = f'https://api.nytimes.com/svc/books/v3/lists/{week}/combined-print-and-e-book-fiction.json?api-key={api_key}'
        response = return_response(url)

        if ('status' in response and response['status']=='OK'):
            book = response['results']['books'][0]
            book = {
                'date': response['results']['published_date'],
                'rank': book['rank'],
                'title': book['title'],
                'author': book['author'],
                'primary_isbn10': book['primary_isbn10'],
                'dagger': book['dagger'],
            }
            month = book['date'].split("-")[0]+'-'+book['date'].split("-")[1]
            if month in all_responses:
                all_responses[month].append(book)
            else:
                all_responses[month] = [book]

            print(f'Response for week {week} added to the list.')

        else:
            print(week)

    json.dump(all_responses, file, indent=2)


Response for week 2011-02-13 added to the list.
Response for week 2011-02-20 added to the list.
Response for week 2011-02-27 added to the list.
Response for week 2011-03-06 added to the list.
Response for week 2011-03-13 added to the list.
2011-03-20
2011-03-27
2011-04-03
2011-04-10
2011-04-17
2011-04-24
2011-05-01
2011-05-08
2011-05-15
2011-05-22
2011-05-29
2011-06-05
2011-06-12
2011-06-19
2011-06-26
2011-07-03
2011-07-10
2011-07-17
2011-07-24
2011-07-31
2011-08-07
2011-08-14
2011-08-21
2011-08-28
2011-09-04
2011-09-11
2011-09-18
2011-09-25
2011-10-02
2011-10-09


KeyboardInterrupt: 

In [30]:
start_date_str = '2011-02-13'
end_date_str = '2023-12-10'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

# Loop through the date range, construct URLs, make requests, and accumulate responses
all_responses = []

output_filename = 'all_responses.jsonl'
with open(output_filename, 'w', encoding='utf-8') as file:
    for current_date in generate_date_range(start_date, end_date):
        week = current_date.strftime('%Y-%m-%d')
        url = f'https://api.nytimes.com/svc/books/v3/lists/{week}/combined-print-and-e-book-fiction.json?api-key={api_key}'
        response = return_response(url)

        all_responses.append(response)
        json.dump(response, file)
        file.write('\n')
        print(f'Response for week {week} added to the list.')


Response for week 2011-02-13 added to the list.
Response for week 2011-02-20 added to the list.
Response for week 2011-02-27 added to the list.
Response for week 2011-03-06 added to the list.
Response for week 2011-03-13 added to the list.
Response for week 2011-03-20 added to the list.
Response for week 2011-03-27 added to the list.
Response for week 2011-04-03 added to the list.
Response for week 2011-04-10 added to the list.
Response for week 2011-04-17 added to the list.
Response for week 2011-04-24 added to the list.
Response for week 2011-05-01 added to the list.
Response for week 2011-05-08 added to the list.
Response for week 2011-05-15 added to the list.
Response for week 2011-05-22 added to the list.
Response for week 2011-05-29 added to the list.
Response for week 2011-06-05 added to the list.
Response for week 2011-06-12 added to the list.
Response for week 2011-06-19 added to the list.
Response for week 2011-06-26 added to the list.
Response for week 2011-07-03 added to th

In [31]:
output_filename = 'all_responses.jsonl'
with open(output_filename, 'w', encoding='utf-8') as file:
    json.dump(all_responses, file, ensure_ascii=False, indent=2)


In [36]:
aggregated_data = defaultdict(lambda: {'count': 0, 'publishers': set()})

with open(output_filename, 'r', encoding='utf-8') as jsonl_file:
    for line in jsonl_file:
        print(line)
        break
        # Load JSON data from each line
        data = json.loads(line)

        # Extract information for each book in the response
        books = data.get('results', {}).get('books', [])

        # Update the count and publisher for each book
        for book in books:
            title = book.get('title', 'Unknown Title')
            aggregated_data[title]['count'] += 1
            aggregated_data[title]['publishers'].add(book.get('publisher', 'Unknown Publisher'))

aggregated_data = {key: {'count': value['count'], 'publishers': list(value['publishers'])} for key, value in aggregated_data.items()}

output_json_file = 'all_books.json'
# Write the aggregated data to a new JSON file
with open(output_json_file, 'w', encoding='utf-8') as json_file:
    json.dump(aggregated_data, json_file, indent=2)


[

